In [1]:
%matplotlib auto
# %matplotlib auto
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import cv2

import sys
import os.path
print(os.getcwd())
sys.path.insert(0, os.getcwd()+"/../../../../libs/tacx")
print(os.getcwd()+"/../../../../libs/tacx")
from SEMContour import *
from MxpStage import MxpStageXmlParser
sys.path.insert(0, os.getcwd()+"/../../../../libs/imutil")
from ImGUI import imread_gray
sys.path.insert(0, os.getcwd()+"/../../../../libs/common")
from PlotConfig import *
from FileUtil import gpfs2WinPath

CWD = r'C:\Localdata\D\Note\Python\apps\MXP\ContourSelect\samplejob\h\cache\dummydb\result\MXP\job1'
CWD = gpfs2WinPath(CWD)

inxml = os.path.join(CWD, r'contourselcal430out.xml')

def loadPatternData(imgfile='', contourfile=''):
    bSucceedReadCt, bSucceedReadIm = False, False

    # read contour
    contour = SEMContour()
    bSucceedReadCt = contour.parseFile(contourfile)
    if not bSucceedReadCt:
        raise OSError("Error, contourfile('{}') cannot be parsed".format(contourfile))
        
    im = None
    try:  # read image
        im, _ = imread_gray(imgfile)
        im = (im/65535).astype(np.float32)
        im = cv2.cvtColor(im, cv2.COLOR_GRAY2BGR)
    except:
        # raise
        pass
    return im, contour

def getContourClassifierData(inxml):
    ocf_parser = MxpStageXmlParser(inxml) #, 'outxml'
    # icf = icf_parser.icf
    df_ocf = ocf_parser.occfs2df()
    patternid = 461
    df_occf = df_ocf.loc[df_ocf.name==patternid, :]
    sr_occf = pd.Series(df_occf.values.flatten(), index=df_occf.columns)
    contourfile = sr_occf.loc['contour/path']
    contourfile = CWD + '/' + contourfile
    imgfile = sr_occf.loc['image/path']
    imgfile = CWD + '/' + imgfile
    im, contour = loadPatternData(imgfile, contourfile)
    return im, contour
    # TODO, add bbox plot
    # df.filter(regex='bbox/outlier', axis=1)
im, contour = getContourClassifierData(inxml)

Using matplotlib backend: Qt5Agg
C:\Localdata\D\Note\Python\apps\MXP\ContourSelect\unittest
C:\Localdata\D\Note\Python\apps\MXP\ContourSelect\unittest/../../../../libs/tacx


MxpStage DEBUG   : ocf tag: result
MxpStage DEBUG   : No summary tag in input xml： C:\Localdata\D\Note\Python\apps\MXP\ContourSelect\samplejob\h\cache\dummydb\result\MXP\job1\contourselcal430out.xml


Warning, can't find 'gpfs' in input file: C:\Localdata\D\Note\Python\apps\MXP\ContourSelect\samplejob\h\cache\dummydb\result\MXP\job1
bbox in origin image: [189, 189, 835, 835]


In [32]:
# plot the SEM image, contour and angle
def plot_image_contour_angle(im, contour, patternid='', arrow_length=1):
    df = contour.toDf()
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_aspect('equal')
    #imw, imh = contour.getshape()
    #ax.set_xlim([0, imw])
    #ax.set_ylim([0, imh])
    xini, yini, xend, yend = contour.getBBox()
    ax.set_xlim([xini, xend])
    ax.set_ylim([yini, yend])
    ax.set_title("Pattern "+patternid+ " image Contour")
    
    # plot image
    ax.imshow(im)
    
    # plot contour
    ax.plot(df.loc[:, 'offsetx'], df.loc[:, 'offsety'], 'b.')
    ax.plot(250.480209, 715.985352, 'r.')
    
    # plot angle
    for _, row in df.iterrows():
        x, y = row.loc['offsetx'], row.loc['offsety']
        angle = row.loc['angle']
        dx, dy = arrow_length*np.cos(angle), arrow_length*np.sin(angle)
        ax.arrow(x, y, dx, dy, width=0.1, fc='y', ec='y') # ,shape='right', overhang=0
        
    plt.gca().invert_yaxis()
    plt.show()
plot_image_contour_angle(im, contour, '461')

In [24]:
# plot contour classifier image, contour by classifers
def plotContourClassifier(im, contour, wndname=''):
    # plot image and classified contour point
    fig = plt.figure()
    ax = fig.add_subplot(111)
    
    imw, imh = contour.getshape()
    ax.set_aspect('equal')
    ax.set_xlim([0, imw])
    ax.set_ylim([0, imh])
    ax.set_title(wndname)

    df = contour.toDf()
    UserGood = df.UserLabel == 1
    FP = (df.UserLabel==1) & (df.clf_label==0)
    FN = (df.UserLabel==0) & (df.clf_label==1)
    ax.imshow(im)
    ax.plot(df.loc[UserGood ,'offsetx'], df.loc[UserGood, 'offsety'], 'b.', markersize=1, label='User Label as good')
    ax.plot(df.loc[~UserGood ,'offsetx'], df.loc[~UserGood, 'offsety'], 'r.', markersize=2, label='User Label as bad'),
    ax.plot(df.loc[FP ,'offsetx'], df.loc[FP, 'offsety'], alpha=0.5,
            linestyle='None', marker= 'o', markersize=4, markeredgewidth=1, markerfacecolor='none', 
            label='FP, UserLable=1 & ClfLabel=0: {}'.format(len(df.loc[FP,:])))
    ax.plot(df.loc[FN ,'offsetx'], df.loc[FN, 'offsety'], alpha=0.5,
            linestyle='None', marker= 'o', markersize=4, markeredgewidth=1, markerfacecolor='none', 
            label='FN, UserLable=0 & ClfLabel=1: {}'.format(len(df.loc[FN,:])))
    #ax = plt.gca() # gca() function returns the current Axes instance
    #ax.set_ylim(ax.get_ylim()[::-1]) # reverse Y
    plt.gca().invert_yaxis()
    plt.legend()
    plt.show()

In [29]:
leftArr = [0.567468, 0.544720, 0.514374, 0.482713, 0.453921, 0.429822, 0.410495, 0.395181, 0.383001, 0.373297, 0.365664, 0.359890, 0.355877, 0.353493, 0.352375, 0.351882, 0.351298, 0.350130, 0.348290, 0.346071, 0.343927, 0.342212, 0.340987, 0.340029, 0.339040, 0.337902, 0.336737, 0.335696, 0.334715, 0.333543, 0.331913, 0.329689, 0.326884, 0.323622, 0.320140, 0.316806, 0.314152, 0.312876, 0.313835, 0.318016, 0.326492, 0.340317, 0.360304, 0.386573, 0.417828, 0.450646]
leftArr = leftArr[-1::-1]
Ig = 0.575737 
rightArr = [0.564595, 0.533566, 0.487334, 0.434199, 0.382961, 0.340042, 0.308248, 0.287320, 0.275337, 0.269955, 0.269012, 0.270665, 0.273422, 0.276220, 0.278512, 0.280247, 0.281684, 0.283127, 0.284718, 0.286387, 0.287908, 0.289012, 0.289534, 0.289573, 0.289473, 0.289590, 0.289999, 0.290436, 0.290524, 0.290072, 0.289171, 0.288098, 0.287167, 0.286580, 0.286317, 0.286130, 0.285696, 0.284855, 0.283788, 0.282952, 0.282858, 0.283905, 0.286517, 0.291533, 0.300620, 0.316334, 0.341522, 0.377962, 0.424629]
Arr = leftArr + [Ig] + rightArr
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(np.arange(0, len(leftArr)), leftArr, label='leftArr')
ax.plot([len(leftArr)], [Ig], 'r.', label='ridge Ig')
ax.plot(np.arange(len(leftArr)+1, len(leftArr)+len(rightArr)+1), rightArr, 'g-', label='rightArr')
plt.legend()
plt.show()

In [16]:
leftArr = [0.567468, 0.544720, 0.514374, 0.482713, 0.453921, 0.429822, 0.410495, 0.395181, 0.383001, 0.373297, 0.365664, 0.359890, 0.355877, 0.353493, 0.352375, 0.351882, 0.351298, 0.350130, 0.348290, 0.346071, 0.343927, 0.342212, 0.340987, 0.340029, 0.339040, 0.337902, 0.336737, 0.335696, 0.334715, 0.333543, 0.331913, 0.329689, 0.326884, 0.323622, 0.320140, 0.316806, 0.314152, 0.312876, 0.313835, 0.318016, 0.326492, 0.340317, 0.360304, 0.386573, 0.417828, 0.450646]
leftArr = leftArr[-1::-1]
leftArr

[0.450646,
 0.417828,
 0.386573,
 0.360304,
 0.340317,
 0.326492,
 0.318016,
 0.313835,
 0.312876,
 0.314152,
 0.316806,
 0.32014,
 0.323622,
 0.326884,
 0.329689,
 0.331913,
 0.333543,
 0.334715,
 0.335696,
 0.336737,
 0.337902,
 0.33904,
 0.340029,
 0.340987,
 0.342212,
 0.343927,
 0.346071,
 0.34829,
 0.35013,
 0.351298,
 0.351882,
 0.352375,
 0.353493,
 0.355877,
 0.35989,
 0.365664,
 0.373297,
 0.383001,
 0.395181,
 0.410495,
 0.429822,
 0.453921,
 0.482713,
 0.514374,
 0.54472,
 0.567468]

In [ ]:
plotContourClassifier(im, contour, 'Pattern 461')

In [ ]:
def plotAllContourClfData(inxml):
    ocf_parser = MxpStageXmlParser(inxml) #, 'outxml'
    # icf = icf_parser.icf
    df_ocf = ocf_parser.occfs2df()
    for _, row in df_ocf.iterrows():
        patternid = row.loc['name']
        usage = row.loc['usage']
        print("{} {}".format(patternid, usage))
        
        contourfile = row.loc['contour/path']
        contourfile = CWD + '/' + contourfile
        imgfile = row.loc['image/path']
        imgfile = CWD + '/' + imgfile
        im, contour = loadPatternData(imgfile, contourfile)
        plotContourClassifier(im, contour, "Pattern "+str(patternid))

In [ ]:
plotAllContourClfData(inxml)